In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [ ]:
df_flow = pd.read_csv("/home/swati/Downloads/cleaning1/3532/flow.tsv" ,delimiter= "\t",header = None)
df_speed = pd.read_csv("/home/swati/Downloads/cleaning1/3532/speed.tsv",delimiter= "\t",header = None)
df_occupancy = pd.read_csv("/home/swati/Downloads/cleaning1/3532/occupancy.tsv",delimiter= "\t",header = None)


In [ ]:
print(df_flow.isnull().values.any())
print(df_speed.isnull().values.any())
print(df_occupancy.isnull().values.any())

In [ ]:
c_bind_lane1 = pd.concat([df_flow.ix[:,0] ,df_speed.ix[:,0] ,df_occupancy.ix[:,0]],axis =1)
c_bind_lane2 = pd.concat([df_flow.ix[:,1] ,df_speed.ix[:,1] ,df_occupancy.ix[:,1]],axis =1)
c_bind_lane3 = pd.concat([df_flow.ix[:,2] ,df_speed.ix[:,2] ,df_occupancy.ix[:,2]],axis =1)
c_bind_lane4 = pd.concat([df_flow.ix[:,3] ,df_speed.ix[:,3] ,df_occupancy.ix[:,3]],axis =1)
c_bind_lane5 = pd.concat([df_flow.ix[:,4] ,df_speed.ix[:,4] ,df_occupancy.ix[:,4]],axis =1)
c_bind_lane6 = pd.concat([df_flow.ix[:,5] ,df_speed.ix[:,5] ,df_occupancy.ix[:,5]],axis =1)


In [ ]:
df_vector1 = pd.concat([c_bind_lane1 ,c_bind_lane2 ,c_bind_lane3])

In [ ]:
df_vector2 = pd.concat([c_bind_lane4, c_bind_lane5, c_bind_lane6])

In [ ]:
df_vector = pd.concat([df_vector1, df_vector2])

In [ ]:
c_bind_lane1.columns = ['flow','speed','occupancy']
c_bind_lane2.columns = ['flow','speed','occupancy']
c_bind_lane3.columns = ['flow','speed','occupancy']

In [ ]:
c_bind_lane4.columns = ['flow','speed','occupancy']
c_bind_lane5.columns = ['flow','speed','occupancy']
c_bind_lane6.columns = ['flow','speed','occupancy']

In [ ]:
df_vector.head()

In [ ]:
df_vector = df_vector.dropna()

In [ ]:
print(df_vector.isnull().values.any())

In [ ]:
df_vector.to_csv("/home/swati/Downloads/mv_3532.csv")

In [ ]:
df = pd.read_csv("/home/swati/Downloads/mv_3532.csv", delimiter = ',')


In [ ]:
counts = df.groupby(['flow', 'speed','occupancy']).count()

In [ ]:
counts.to_csv("/home/swati/Downloads/counts_3532.csv", delimiter = ',')

In [ ]:
df = pd.read_csv("/home/swati/Downloads/counts_3532.csv", delimiter = ',')

In [ ]:
Total = df['Count'].sum()

In [ ]:
df['relative_Frequency'] = df['Count'].apply(lambda x : x/Total)

In [ ]:
df_matrix = df.as_matrix(columns=df.columns[0:3])

In [ ]:
nbrs = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(df_matrix)

In [ ]:
distances, indices = nbrs.kneighbors(df_matrix)

In [ ]:
df_matrix_including_rel_freq = df.as_matrix(columns=df.columns[0:5])


In [ ]:
for row in range(0,len(distances)) :
    num_of_neighbors = 20
    weighted_freq_sum = 0
    for neigh in range(1,num_of_neighbors):
        
        distance = distances[row][neigh]
        index_of_neigh = indices[row][neigh]
        relative_freuency_of_neighbor = df_matrix_including_rel_freq[index_of_neigh][4]
        weighted_freq_sum = weighted_freq_sum + (relative_freuency_of_neighbor/distance)
    
   
    df.ix[row,'weighted_freq_sum'] = weighted_freq_sum    
    df.ix[row,'Total'] =  df.ix[row,'relative_Frequency']+ (weighted_freq_sum /20)

In [ ]:
df = df.sort(columns = ['Total'],)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.ix[0, 'LL'] = 1
df.ix[0, 'UL'] = 2

In [ ]:
for i in range(1,len(df)):
        df.ix[i, 'LL'] = df.ix[i-1, 'UL'] + 1
        df.ix[i, 'UL'] = df.ix[i, 'LL'] + (df.ix[i, 'Count'] - 1)

In [ ]:
final = pd.DataFrame()

In [ ]:
for i in range(0,Total,100):
    row = df[(df['LL'] <= i) & (df['UL'] >= i)]
    row2 = row.drop(row.columns[[3,4,5,7,8]],axis=1)
    final = final.append(row2) 

In [ ]:
final['probability'] = final['Total'].apply(lambda x : format(x,'.8f'))

In [ ]:
final.to_csv("/home/swati/Downloads/3532.csv")